In [98]:
import pandas as pd
import openpyxl
from openpyxl import load_workbook

# Charger les données
df = pd.read_excel(r"C:\Users\Cellphone\Downloads\transactions_rawbank_fictif.xlsx")

# Affichage des premières lignes
df.head(10)

,ID_Transaction,Date,Agence,Type_Operation,Client,Montant
0,T1000,2025-04-01,Ngaliema,Paiement,Client D,466298
1,T1001,2025-04-02,Limete,Paiement,Client A,387393
2,T1002,2025-04-03,Ngaliema,Paiement,Client C,35754
3,T1003,2025-04-04,Ngaliema,Virement,Client B,227814
4,T1004,2025-04-05,Limete,Virement,Client A,238714
5,T1005,2025-04-06,Limete,Virement,Client A,143483
6,T1006,2025-04-07,Ngaliema,Dépôt,Client A,134983
7,T1007,2025-04-08,Gombe,Paiement,Client C,461486
8,T1008,2025-04-09,Ngaliema,Virement,Client B,215835
9,T1009,2025-04-10,Ngaliema,Virement,Client A,402896


In [99]:
# Vérifier s'il y a des valeurs manquantes 
print(df.isnull().sum())

ID_Transaction    0
Date              0
Agence            0
Type_Operation    0
Client            0
Montant           0
dtype: int64


In [100]:
# Total par type d'opération 
operations = df.groupby('Type_Operation')['Montant'].sum().sort_values(ascending=False)
operations

Type_Operation
Virement    8574456
Paiement    6866256
Dépôt       4886308
Retrait     4298103
Name: Montant, dtype: int64

In [101]:
# Nombre de transaction par agence 
transactions_agences = df.groupby('Agence')['Agence'].value_counts().sort_values(ascending=False).reset_index()
print(transactions_agences)


     Agence  count
0     Gombe     36
1    Limete     33
2  Ngaliema     31


In [118]:
# Total des montants par agence
montant_par_agence= df.groupby('Agence')['Montant'].sum().sort_values(ascending=False).reset_index()
montant_par_agence

,Agence,Montant
0,Gombe,8995391
1,Ngaliema,8161624
2,Limete,7468108


In [103]:
# Type d'operation selon les agences

t_o = df.groupby(['Agence','Type_Operation'])['Montant'].sum()
t_o

Agence    Type_Operation
Gombe     Dépôt             2118425
          Paiement          1971470
          Retrait           1487643
          Virement          3417853
Limete    Dépôt             1186897
          Paiement          2312210
          Retrait           2127804
          Virement          1841197
Ngaliema  Dépôt             1580986
          Paiement          2582576
          Retrait            682656
          Virement          3315406
Name: Montant, dtype: int64

In [111]:
# Nombre d'opération par jour 
df['Date'] = pd.to_datetime(df['Date'],errors='coerce')
operations_par_jour = df.groupby(df['Date'].dt.date).size()
operations_par_jour

Date
2025-04-01    1
2025-04-02    1
2025-04-03    1
2025-04-04    1
2025-04-05    1
             ..
2025-07-05    1
2025-07-06    1
2025-07-07    1
2025-07-08    1
2025-07-09    1
Length: 100, dtype: int64

In [125]:
# Nombre d'operation par mois 
operations_par_mois = df.groupby(df['Date'].dt.strftime('%B')).size()
operations_par_mois

Date
April    30
July      9
June     30
May      31
dtype: int64

In [126]:
# Client le plus actif (avec plus d'opérations)
client_frequent = df['Client'].value_counts().idxmax()
client_frequent

'Client D'

In [120]:
#Volume par type d'opérations
volume_par_type = df['Type_Operation'].value_counts()
volume_par_type

Type_Operation
Virement    34
Paiement    25
Dépôt       22
Retrait     19
Name: count, dtype: int64

In [123]:
# Enregistrer un rapport Excel propre avec resumé

with pd.ExcelWriter("analyse_banquier.xlsx",engine = "openpyxl") as writer :
    df.to_excel(writer,sheet_name='Données',index=False)
    # Resumé des opérations 
    operations_df = operations.reset_index()
    operations_df.columns= ["Type d'opération","Total Montant"]
    operations_df.to_excel(writer,sheet_name='Resumé',index=False)
    volume_par_type.to_excel(writer,sheet_name='Volume par type',index=False)
    montant_par_agence.to_excel(writer,sheet_name='Montant par Agence')
    operations_par_jour.to_excel(writer,sheet_name='Ops_par_jour')
    pd.DataFrame({'Client le pus actif':[client_frequent]}).to_excel(writer,sheet_name='Client_actif')
    
    

wb= load_workbook("rapport_banque_nettoyé.xlsx")
wb.save("rapport_banque_nettoyé.xlsx")
print("Analyse terminée. Resultats enregistrés dans 'analyse_banquier.xlsx'")


Analyse terminée. Resultats enregistrés dans 'analyse_banquier.xlsx'
